[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/TotoroUI-jupyter/blob/main/ipadapter.ipynb)

In [ ]:
!pip install -q torchsde einops diffusers accelerate xformers==0.0.25
!apt -y install -qq aria2

%cd /content
!git clone -b totoro https://github.com/camenduru/ComfyUI /content/TotoroUI
!git clone -b totoro https://github.com/camenduru/ComfyUI_IPAdapter_plus /content/TotoroUI/IPAdapter

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/128713 -d /content/TotoroUI/models -o dreamshaper_8.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors -d /content/TotoroUI/models/clip_vision -o CLIP-ViT-H-14-laion2B-s32B-b79K.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.safetensors -d /content/TotoroUI/models/ipadapter -o ip-adapter_sd15.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_light_v11.bin -d /content/TotoroUI/models/ipadapter -o ip-adapter_sd15_light_v11.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.safetensors -d /content/TotoroUI/models/ipadapter -o ip-adapter-plus_sd15.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.safetensors -d /content/TotoroUI/models/ipadapter -o ip-adapter-plus-face_sd15.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-full-face_sd15.safetensors -d /content/TotoroUI/models/ipadapter -o ip-adapter-full-face_sd15.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_vit-G.safetensors -d /content/TotoroUI/models/ipadapter -o ip-adapter_sd15_vit-G.safetensors

# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/RunDiffusion/Juggernaut-XL-v9/resolve/main/Juggernaut-XL_v9_RunDiffusionPhoto_v2.safetensors -d /content/TotoroUI/models -o Juggernaut-XL_v9_RunDiffusionPhoto_v2.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/image_encoder/model.safetensors -d /content/TotoroUI/models/clip_vision -o CLIP-ViT-bigG-14-laion2B-39B-b160k.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl_vit-h.safetensors  -d /content/TotoroUI/models/ipadapter -o ip-adapter_sdxl_vit-h.safetensors 
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus_sdxl_vit-h.safetensors  -d /content/TotoroUI/models/ipadapter -o ip-adapter-plus_sdxl_vit-h.safetensors 
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus-face_sdxl_vit-h.safetensors  -d /content/TotoroUI/models/ipadapter -o ip-adapter-plus-face_sdxl_vit-h.safetensors 
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl.safetensors -d /content/TotoroUI/models/ipadapter -o ip-adapter_sdxl.safetensors

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/camenduru/TotoroUI-jupyter/assets/54370274/8744d2de-e74b-4b68-b46d-e3a8bd649cdf -d /content -o ip1.png
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/camenduru/TotoroUI-jupyter/assets/54370274/e42ff968-2298-4a65-b685-1e9c0c0eb2cf -d /content -o ip2.png
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/camenduru/TotoroUI-jupyter/assets/54370274/27a47c1a-085e-4675-bf21-9075066796b4 -d /content -o mask.png
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/camenduru/TotoroUI-jupyter/assets/54370274/2fce96a5-2d85-42fc-8841-7375323149e5 -d /content -o mask1.png
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/camenduru/TotoroUI-jupyter/assets/54370274/5217c416-69e5-42ab-a402-23702371017f -d /content -o mask2.png

%cd /content/TotoroUI
import torch
import numpy as np
from PIL import Image
import totoro
import nodes
import sys
sys.path.append('/content/TotoroUI/IPAdapter')
with torch.no_grad():
  model_patcher, clip, vae, clipvision = totoro.sd.load_checkpoint_guess_config("/content/TotoroUI/models/dreamshaper_8.safetensors", output_vae=True, output_clip=True, embedding_directory=None)

import IPAdapterPlus
IPAdapterPlus_model = IPAdapterPlus.IPAdapterUnifiedLoader().load_models(model_patcher, 'PLUS (high strength)', lora_strength=0.0, provider="CPU", ipadapter=None)

output1_image, output1_mask = nodes.LoadImage().load_image("/content/ip1.png")
output2_image, output2_mask = nodes.LoadImage().load_image("/content/ip2.png")
mask1_image, mask1_mask = nodes.LoadImage().load_image("/content/mask1.png")
mask2_image, mask2_mask = nodes.LoadImage().load_image("/content/mask2.png")
ip_model_patcher1 = IPAdapterPlus.IPAdapterSimple().apply_ipadapter(IPAdapterPlus_model[0], IPAdapterPlus_model[1], output1_image, 0.7, 0.0, 1.0, 'standard', attn_mask=mask1_mask)
# ip_model_patcher2 = IPAdapterPlus.IPAdapterSimple().apply_ipadapter(ip_model_patcher1[0], IPAdapterPlus_model[1], output2_image, 0.7, 0.0, 1.0, 'standard', attn_mask=mask2_mask)

In [ ]:
latent = {"samples":torch.zeros([1, 4, 512 // 8, 512 // 8])}

tokens = clip.tokenize("a closeup of a woman astronaut in an enchanted forest, high quality, detailed")
cond, pooled = clip.encode_from_tokens(tokens, return_pooled=True)
cond = [[cond, {"pooled_output": pooled}]]

n_tokens = clip.tokenize("blurry, lowres, bad art, ill, distorted, malformed, horror")
n_cond, n_pooled = clip.encode_from_tokens(n_tokens, return_pooled=True)
n_cond = [[n_cond, {"pooled_output": n_pooled}]]

sample = nodes.common_ksampler(model=ip_model_patcher1[0], 
                          seed=3, 
                          steps=30, 
                          cfg=7.0, 
                          sampler_name="dpmpp_2m", 
                          scheduler="karras", 
                          positive=cond, 
                          negative=n_cond,
                          latent=latent, 
                          denoise=1)

with torch.inference_mode():
  sample = sample[0]["samples"].to(torch.float16)
  vae.first_stage_model.cuda()
  decoded = vae.decode_tiled(sample).detach()

Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0])